## Limpieza de textos

Vamos a eliminar genero y carácteres redundantes de la columna descripcion, además de preprocesar y tokenizar el texto para facilitar la lectura al modelo

Cambios: 

 Eliminar URL (realizando antes)<br>
 Eliminar números (realizando antes)<br>
 Eliminar correos (realizando antes) <br> <br>
 Traducción de textos en otras lenguas al castellano. <br>
 Convertir a minúsculas<br>
 Eliminar signos de puntuación<br>
 Lematizar texto<br>
 Detectar y eliminar patrones tipo /a, /as

In [ ]:
import csv
from googletrans import Translator

input_file = 'ofertasAnonimizado_unique_ts.csv'
output_file = 'ofertasAnonimizado_unique_ts_traducido.csv'
provincias_catalanas = {'Barcelona', 'Girona', 'Tarragona', 'Lleida'}

translator = Translator()

with open(input_file, 'r', encoding='utf-8') as fin, open(output_file, 'w', encoding='utf-8', newline='') as fout:
    reader = csv.DictReader(fin, delimiter=';')
    fieldnames = reader.fieldnames
    writer = csv.DictWriter(fout, fieldnames=fieldnames, delimiter=';')
    writer.writeheader()
    for row in reader:
        region = row.get('region', '')
        if region in provincias_catalanas:
            desc = row.get('descripcion', '')
            # Detectar si hay catalán (heurística simple: palabras típicas)
            if any(pal in desc.lower() for pal in ['amb', 'per', 'dilluns', 'divendres', 'dimecres', 'dijous', 'dissabte', 'diumenge', 'treballar', 'ofereix', 'incorporació', 'contracte', 'empresa', 'persones', 'serveis', 'activitat', 'atenció', 'domiciliària', 'grans', 'català', 'castellà', 'client', 'oferta', 'inscripció', 'referència', 'convocatòria', 'col·lectiu', 'conveni', 'salarial', 'torn', 'matí', 'tarda', 'horari', 'dilluns', 'divendres', 'dimecres', 'dijous', 'dissabte', 'diumenge']):
                try:
                    translated = translator.translate(desc, src='ca', dest='es').text
                    row['descripcion'] = translated
                except Exception:
                    pass  # Si falla la traducción, deja el texto original
        writer.writerow(row)
print('Traducción completada')


## sentiment analysis
Sobre la columna descripcion vamos a usar distintos modelos que luego haremos usaremos para un voting clasifier.

Nuestros modelos: <br>
Vader de nltk, bastante simple y rápido para procesar texto<br>
TextBlob, también un modelo sencillo para procesar sentimientos<br>
BERT, modelo basado en transformers más complejo, debería dar un desempeño mejor, por eso consideramos más importancia a este modelo en las votaciones

In [ ]:
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline

nltk.download('vader_lexicon') # descargar el modelo para usar el sentiment analysis

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\alvar\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
sia = SentimentIntensityAnalyzer()

bert_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# scores por clase
def vader_scores(text):
    scores = sia.polarity_scores(text)
    return {
        "neg": scores["neg"],
        "neu": scores["neu"],
        "pos": scores["pos"]
    }

def textblob_scores(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return {"neg": 0.0, "neu": 1 - polarity, "pos": polarity}
    elif polarity < 0:
        return {"neg": abs(polarity), "neu": 1 - abs(polarity), "pos": 0.0}
    else:
        return {"neg": 0.0, "neu": 1.0, "pos": 0.0}

def bert_scores(text):
    result = bert_pipeline(text)[0]['label']
    # convertir estrellas a valores aproximados
    if "1" in result:
        return {"neg": 0.9, "neu": 0.1, "pos": 0.0}
    elif "2" in result:
        return {"neg": 0.7, "neu": 0.3, "pos": 0.0}
    elif "3" in result:
        return {"neg": 0.2, "neu": 0.6, "pos": 0.2}
    elif "4" in result:
        return {"neg": 0.0, "neu": 0.2, "pos": 0.8}
    elif "5" in result:
        return {"neg": 0.0, "neu": 0.1, "pos": 0.9}


# Ensemble ponderado
def weighted_sentiment(text, weights=(0.3, 0.3, 0.4)):
    vader_w, blob_w, bert_w = weights
    
    v = vader_scores(text)
    t = textblob_scores(text)
    b = bert_scores(text)
    
    combined = {
        "neg": vader_w*v["neg"] + blob_w*t["neg"] + bert_w*b["neg"],
        "neu": vader_w*v["neu"] + blob_w*t["neu"] + bert_w*b["neu"],
        "pos": vader_w*v["pos"] + blob_w*t["pos"] + bert_w*b["pos"]
    }
    
    # clasificar
    final_label = max(combined, key=combined.get)
    label_map = {"neg": -1, "neu": 0, "pos": 1}
    return label_map[final_label]


c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alvar\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [12]:
df.head()

,id,timestamp,titulo,ocupacion,descripcion,provincia,tipo_contrato
0,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,AGENTE COMERCIAL DE SEGUROS (REF.: 6891),corredor de seguros/corredora de seguros,TAREAS:Prospección de nuevos asegurados/as.Pla...,Asturias,Contrato
1,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PERSONAL CONDUCCIÓN DE CAMIONES RÍGIDOS Y GÓND...,Conductor de vehículo de carga/conductora de v...,DESCRIPCIÓN: Se necesita cubrir cuatro puestos...,Huesca,Contrato
2,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,EDUCADORES SOCIALES,Trabajador social/trabajadora social,EDUCADOR/A SOCIAL PARA HOGAR EN ARINAGA. FINES...,Las Palmas,Determinado
3,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PIZZERO/A (REF. 042025002051),Pizzero/pizzera,FUNCIONES: Elaboración de pizzas REQUISITOS: 2...,Islas Baleares,Determinado
4,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,INTÉRPRETES DE LA LENGUA DE SIGNOS,intérprete de lengua de signos,INTÉRPRETE DE LENGUA DE SIGNOS PARA PUESTOS EN...,Santa Cruz de Tenerife,Determinado


In [ ]:

df = pd.read_csv("../empleos-espanoles-eures-2025.csv", sep=";") # cambiar el dataset a su ultima version traducida y tokenizada
df["sentimiento"] = df["descripcion"].apply(weighted_sentiment)

# -----------------------------
# 6. Mostrar resultado
# -----------------------------
print(df)
